In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torch import device
from tqdm.notebook import tqdm
from ResNet import ResNet18
torch.manual_seed(47)
np.random.seed(47)


In [ ]:
data = torchvision.datasets.CIFAR100(
    root="./data", train=False, download=True, transform=tr
)
tr = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
batch_size = 32
datal = torch.utils.data.DataLoader(data, batch_size=batch_size*2, shuffle=False, num_workers=2, )

# Neural Collapse

### separating the data per class

In [ ]:
def load_model(path):
    d = torch.load(path)
    d["cl.weight"] = d["model.13.weight"]
    d["cl.bias"] = d["model.13.bias"]
    d.pop("model.13.weight")
    d.pop("model.13.bias")
    resnet = ResNet18(64,2,100).to(device)
    resnet.load_state_dict(d)
    return resnet
resnet = load_model("/content/drive/MyDrive/resnet_360_epoch.pth")

In [ ]:
#load the model
tab = []
for epoch in range(360,361,10):# CHange this
  path = f"/content/drive/MyDrive/resnet_360_epoch.pth"
  # resnet = ResNet18(64,2,100).to(device)
  # resnet.load_state_dict(torch.load(path))
  resnet.eval()
  features ={i:[] for i in range(100)}
  with torch.no_grad():
    for inputs, labels in tqdm(datal):
        inputs = inputs.to(device)
        labels = labels.to(device)
        out = resnet.model(inputs)
        for i in range(len(labels)):
            features[labels[i].item()].append(out[i])
  for key in features.keys():
    features[key] = torch.stack(features[key])
  vars = {i:[] for i in range(100)}
  tmp = []


In [ ]:
global_mean_vector = torch.mean(torch.cat(list(features.values())),dim=0)
print(global_mean_vector.shape)
global_var_vector = torch.var(torch.cat(list(features.values())),dim=0)
print(global_var_vector.shape)

#W the last layer weights
weights = resnet.cl.weight.data
print(weights.shape)

## NC 1

variability collapse : In class variations converges towards 0.



In [ ]:
vars = {i:[] for i in range(100)}
class_means = {i:[] for i in range(100)} # globally centered class mean
# tmp_T = []
# tmp_B = []
# tmp_W = []
centered_norms = []
weights
for key in features.keys():
  norm = features[key]  # - global_mean_vector
  class_means[key] = torch.mean(norm,dim=0)
  # vars[key] = torch.var(norm,dim=0)
  centered_norms.append(torch.linalg.vector_norm(class_means[key] - global_mean_vector))

centered_norms = torch.tensor(centered_norms)
#Fig 2.
vals_1 = torch.std(centered_norms) / torch.mean(centered_norms)

class_cos_sim = []
for key in class_means.keys():
  for key2 in class_means.keys():
    if key < key2:
      continue
    norm=  (class_means[key] -global_mean_vector )@ (class_means[key2] - global_mean_vector)
    norm = norm/(torch.linalg.vector_norm(class_means[key] - global_mean_vector)*torch.linalg.vector_norm(class_means[key2] - global_mean_vector)   )

    class_cos_sim.append(norm)
#Fig 3
class_cos_sim = torch.tensor(class_cos_sim)
vals_2 = torch.std(class_cos_sim)
#Fig 4
vals_3 = class_cos_sim + 1/(class_cos_sim.shape[0] - 1)
vals_3 = torch.mean(vals_3)


In [ ]:
print(vals_1)
print(vals_2)
print(vals_3)

In [ ]:
#Fig 5



#Fig 6.




#Fig 7.


# sigma_T =
# sigma_B
# sigma_W

In [ ]:
print(vals_4)

## NC 2

## NC 3


## NC 4

## NC 5